<h2> Automatic filtering </h2> 

In [ ]:
# import internal .py files
import file_path_management as fpath
import public_library as plib

In [ ]:
# import packages
import csv
import pandas as pd
import re
from bs4 import BeautifulSoup
import requests
import os
import numpy as np
import string
import math
from nltk import ngrams

<h3> Parameters: </h3> 

In [ ]:
# on-topic keyword lexicon
on_topic_kws = [
    'thalamocortical', 'thalamo-cortical', 'corticothalamic', 'cortico-thalamic',
    'tracing', 'tracer', 'tract tracing', 'tract-tracing', 'axonal tracing', 'neural tracing', 'anatomical tracing', 'neuroanatomical tracing', 'anatomical neural tracing',
    "staining", "dye",
    'thalamus', 'cortex', 'thalamic', 'cortical',  
    'connection', 'projection', 'connectivity', 'connectome', "anterograde", "retrograde", "injection", "injected", "injecting", "inject"]

on_topic_kws_weights = {
    'tracing': 15, 'tracer': 15, 'tract tracing': 15, 'tract-tracing': 15, 'axonal tracing': 15, 'neural tracing': 15, 
    'anatomical tracing': 15, 'anatomical neural tracing': 15, 'neuroanatomical tracing': 15, "anterograde": 15, "retrograde": 15,
    "injection": 10, "injected": 10, "injecting": 10, "inject": 10, 
    'thalamocortical': 10, 'thalamo-cortical': 10, 'corticothalamic': 10, 'cortico-thalamic': 10, "staining": 10, "dye": 10,
    'connection': 5, 'projection': 5, 'connectivity': 5, 
    'thalamus': 2, 'cortex': 2, 'thalamic': 2, 'cortical': 2, 
    'connectome': 1}

#  pay attention to false negative, false positive cases

if len(on_topic_kws) != len(on_topic_kws_weights):
    raise ValueError("Length of on_topic_kws and on_topic_kws_weights should be the same.")

# ChatGPT, queries for relatedness of topic
ChatGPT_related_queries = ['Does the given text include information of thalamocotical connection?',
                           'Does this paper provide data of thalamocotical connection?',
                           'Does the given text include information of connection between thalamus and cortex?']

<h3> Predefined fucntions: </h3> 

In [ ]:
def info_filling(input_path, output_path, start, end):
    # scan each row in the potential related literature and extract information
    df = pd.read_csv(input_path, header=None, sep=",")
    df.columns = ["DOI", "PMID", "PMCID", "title", "full_text_link", "full_text_source", "pdf_link"]
    
    for ind in range(start, end):
        # check if the full_text_link is one of our websites
        flag = False
        for website in plib.websites:
            if website in df.at[ind, "full_text_source"]:
                flag = True
                break
        if not flag:
            continue

        if df.at[ind, "full_text_link"] == df.at[ind, "full_text_link"]:
            url = str(df.at[ind, "full_text_link"]).strip()
            source = str(df.at[ind, "full_text_source"]).strip()
            info = plib.extract_info_from_webpage(url, source)
        else:
            raise Exception("Error! Full text link is not available!")
        
        # info = {
        #     "doi": doi,
        #     "pmid": pmid,
        #     "pmcid": pmcid,
        #     "title": title,
        #     "abstract": abstract,
        #     "keywords": keywords,
        #     "pdf_link": pdf_link
        # }

        # doi
        if info['doi'] == info['doi'] and df.at[ind, "DOI"] == df.at[ind, "DOI"] and info['doi'] != df.at[ind, "DOI"]:
            print(df.at[ind, "DOI"])
            print(info['doi'])
        if info['doi'] == info['doi']:
            doi = info['doi']
        else:
            doi = df.at[ind, "DOI"]
        
        # pmid
        if info['pmid'] == info['pmid'] and df.at[ind, "PMID"] == df.at[ind, "PMID"] and str(int(info['pmid'])) != str(int(df.at[ind, "PMID"])):
            print(info['pmid'])
            print(df.at[ind, "PMID"])
        if info['pmid'] == info['pmid']:
            pmid = str(int(info['pmid']))
        elif df.at[ind, "PMID"] == df.at[ind, "PMID"]:
            pmid = str(int(df.at[ind, "PMID"]))
        else:
            pmid = np.nan
        
        # pmcid
        if info['pmcid'] == info['pmcid'] and df.at[ind, "PMCID"] == df.at[ind, "PMCID"] and info['pmcid'] != df.at[ind, "PMCID"]:
            print(info['pmcid'])
            print(df.at[ind, "PMCID"])
        if info['pmcid'] == info['pmcid']:
            pmcid = info['pmcid']
        else:
            pmcid = df.at[ind, "PMCID"]
        
        # full_text_link
        if df.at[ind, "full_text_link"] == df.at[ind, "full_text_link"]:
            full_text_link = str(df.at[ind, "full_text_link"]).strip()
        else:
            raise Exception("Error! Full text link is not available!")
        
        # full_text_surce
        if df.at[ind, "full_text_source"] == df.at[ind, "full_text_source"]:
            full_text_surce = str(df.at[ind, "full_text_source"]).strip()
        else:
            raise Exception("Error! full text surce is not available!")
        
        # pdf_link
        if info['pdf_link'] == info['pdf_link']:
            pdf_link = info['pdf_link']
        else:
            pdf_link = df.at[ind, "pdf_link"]
        
        # title
        if info['title'] == info['title']:
            title = info['title']
            title = title.replace(";", ",")
        else:
            title = df.at[ind, "title"]
        
        # abstract
        if info['abstract'] == info['abstract']:
            abstract = info['abstract']
            abstract = ''.join(e for e in abstract if (e.isalpha() or e == " " or e == "-"))
        else:
            # print("Warning! Abstract is not available!", full_text_link)
            abstract = np.nan
        
        # keywords
        if info['keywords'] == info['keywords']:
            keywords = info['keywords']
            keywords = keywords.replace(";", ",")
            keywords = ''.join(e for e in keywords if (e.isalpha() or e == " " or e == "-" or e == ","))
        else:
            keywords = np.nan
    
        columns = ["DOI", "PMID", "PMCID", "full_text_link", "full_text_source", "pdf_link", "Title", "Abstract", "Keywords", "index"]
        row = {
            "DOI": [doi],
            "PMID": [pmid],
            "PMCID": [pmcid],
            "full_text_link": [full_text_link],
            "full_text_source": [full_text_surce],
            "pdf_link": [pdf_link],
            "Title": [title],
            "Abstract": [abstract],
            "Keywords": [keywords],
            "index": [float(0)]
        }
        # print(row)

        if not plib.add_row_to_csv(output_path, row, columns):
            print("Error detected when adding a row to csv!")
        
        print(ind)
# --------------------start of test code--------------------
# input_path = fpath.poten_litera_ids_ftl_filled
# output_path = fpath.poten_litera_litera_db

# # clear file
# plib.clear_file(output_path)

# info_filling(input_path, output_path)
# ---------------------end of test code---------------------

In [ ]:
def count_keyword(text, keyword, keyword_length):
    # remove non-alphabetic characters but keep spaces and "-"
    text = ''.join(e for e in text if (e.isalpha() or e == " " or e == "-"))
    # print(text)
    text = text.strip().lower()
    # print(text)
    
    words = []
    # sentence = 'I have a laptop case and a laptop bag'
    n = keyword_length
    n_grams = ngrams(text.split(), n)
    for gram in n_grams:
        word = gram[0]
        if n > 0:
            for i in range(1, n):
                word = word + " " + gram[i]
        words.append(word)
    
    # print(words)
    
    word_count = 0
    for word in words:
        # print(word)
        if word == keyword:
            word_count += 1
    return word_count
# --------------------start of test code--------------------
# text = 'This apple 6i7s very tasty？、  2but th&e banana this is not delicious at Is all.6'
# keyword = 'this apple'
# count = count_keyword(text, keyword, 2)
# print(count)
# ---------------------end of test code---------------------

In [ ]:
def count_freq_from_liter(text, on_topic_kws, type):
    text = ''.join(e for e in text if (e.isalpha() or e == " " or e == "-"))
    # print(text)
    text = text.strip().lower()
    # print(text)

    text_length = len(text)
    keywords_count = {}
    keywords_fre = {}

    # count the on-topic keywords
    for i in range(len(on_topic_kws)):
        word_count = count_keyword(text, on_topic_kws[i], len(on_topic_kws[i].split()))
        if type == "count":
            keywords_count[on_topic_kws[i]] = word_count
        elif type == "frequency":
            keywords_fre[on_topic_kws[i]] = math.ceil((word_count*100/text_length))/100
        else:
            raise Exception("Error! The only two options for type are 'count' or 'frequency'!")
    
    if type == "count":
        return keywords_count
    elif type == "frequency":
        return keywords_fre
    else:
        raise Exception("Error! The only two options for type are 'count' or 'frequency'!")
# --------------------start of test code--------------------
# text = 'Vision for action: thalamic and cortical inputs to the macaque superior tract neural tracing, parietal lobule The dorsal visual stream, the cortical circuit that in the primate brain is mainly dedicated to the visual control of actions, is split into two routes, a lateral and a medial one, both involved in coding different aspects of sensorimotor control of actions. The lateral route, named "lateral grasping network", is mainly involved in the control of the distal part of prehension, namely grasping and manipulation. The medial route, named "reach-to-grasp network", is involved in the control of the full deployment of prehension act, from the direction of arm movement to the shaping of the hand according to the object to be grasped. In macaque monkeys, the reach-to-grasp network (the target of this review) includes areas of the superior parietal lobule (SPL) that hosts visual and somatosensory neurons well suited to control goal-directed limb movements toward stationary as well as moving objects. After a brief summary of the neuronal functional properties of these areas, we will analyze their cortical and thalamic inputs thanks to retrograde neuronal tracers separately injected into the SPL areas V6, V6A, PEc, and PE. These areas receive visual and somatosensory information distributed in a caudorostral, visuosomatic trend, and some of them are directly connected with the dorsal premotor cortex. This review is particularly focused on the origin and type of visual information reaching the SPL, and on the functional role this information can play in guiding limb interaction with objects in structured and dynamic environments. Area PEc; Area V6; Area V6A; Dorsal visual stream; Goal-directed arm movement; Sensorimotor integration.'
# keywords_count_fre = count_freq_from_liter(text, on_topic_kws, type="count")
# print(keywords_count_fre)
# ---------------------end of test code---------------------

In [ ]:
def calcul_index(keywords_count_or_fre, on_topic_kws_weights):
    index = 0
    for key, value in keywords_count_or_fre.items():
        index += keywords_count_or_fre[key] * keywords_count_or_fre[key]
    return index
# --------------------start of test code--------------------
# keywords_count_or_fre = {}
# on_topic_kws_weights = {}
# index = calcul_related(keywords_count_or_fre, on_topic_kws_weights)
# print(index)
# ---------------------end of test code---------------------

In [ ]:
def weight_and_rank(input_path, output_path, on_topic_kws_weights):
    df = pd.read_csv(input_path, header=None, sep=",")
    df.columns = ["DOI", "PMID", "PMCID", "full_text_link", "full_text_source", "pdf_link", "Title", "Abstract", "Keywords", "Introduction"]

    for ind in df.index:
        text = ""
        text = df["Title"][ind] + " " + df["Abstract"][ind] + " " + df["Keywords"][ind] + " " + df["Introduction"][ind]
        # type = "count"
        type = "frequency"
        keywords_count_or_fre = (text, on_topic_kws, type)
        index = calcul_index(keywords_count_or_fre, on_topic_kws_weights)
        df.at[ind, "index"] = index
        print(ind)
    
    # rank
    df = df.sort_values(by=["index"], ascending=False)
    df.to_csv(output_path, header=True, index=False)
    print("Weighting and ranking the potentially related literature succeded!")
    print("Enjoy reading!")
# --------------------start of test code--------------------
# test code
# ---------------------end of test code---------------------

<h3> Main program: </h3> 

In [ ]:
# step 1: extract and filling text
input_path = fpath.poten_litera_ids_ftl_filled
output_path = fpath.poten_litera_litera_db

# clear file
plib.clear_file(output_path)

info_filling(input_path, output_path, 0, 10769)

In [ ]:
# step 2: assign index to each literature and rank them
input_path = fpath.poten_litera_litera_db
output_path = fpath.poten_litera_litera_db_ranked

# clear file
plib.clear_file(output_path)

weight_and_rank(input_path, output_path, on_topic_kws_weights)

<h3> Next step: manually read papers and find all actually related literature </h3>